<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-preperation" data-toc-modified-id="Data-preperation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data preperation</a></span></li><li><span><a href="#Create-the-network" data-toc-modified-id="Create-the-network-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Create the network</a></span><ul class="toc-item"><li><span><a href="#Compare-Degree-Centrality-&amp;-Page-Rank-&amp;-Betweenness-Centrality" data-toc-modified-id="Compare-Degree-Centrality-&amp;-Page-Rank-&amp;-Betweenness-Centrality-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Compare Degree Centrality &amp; Page Rank &amp; Betweenness Centrality</a></span></li></ul></li></ul></div>

# Bike Network Analysis

## Data preperation

In [2]:
# import modules
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# read data
path_to_data = "../data/NYC-CitiBike-2016.csv"
df = pd.read_csv(path_to_data)
df.columns
df.columns = df.columns.str.replace(" ", "_")
df.head()

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,839,1/1/2016 00:09:55,1/1/2016 00:23:54,532,S 5 Pl & S 4 St,40.710451,-73.960876,401,Allen St & Rivington St,40.720196,-73.989978,17109,Customer,NaN,0
1,686,1/1/2016 00:21:17,1/1/2016 00:32:44,3143,5 Ave & E 78 St,40.776829,-73.963888,3132,E 59 St & Madison Ave,40.763505,-73.971092,23514,Subscriber,1960.0,1
2,315,1/1/2016 00:33:11,1/1/2016 00:38:26,3164,Columbus Ave & W 72 St,40.777057,-73.978985,3178,Riverside Dr & W 78 St,40.784145,-73.983625,14536,Subscriber,1971.0,1
3,739,1/1/2016 00:40:51,1/1/2016 00:53:11,223,W 13 St & 7 Ave,40.737815,-73.999947,276,Duane St & Greenwich St,40.717488,-74.010455,24062,Subscriber,1969.0,1
4,1253,1/1/2016 00:44:16,1/1/2016 01:05:09,484,W 44 St & 5 Ave,40.755003,-73.980144,151,Cleveland Pl & Spring St,40.722104,-73.997249,16380,Customer,NaN,0


In [4]:
df_graph = df.groupby(["start_station_id", "end_station_id"]).size().reset_index()
df_graph.rename(columns={0:"weight"}, inplace=True)
df_graph.head()

,start_station_id,end_station_id,weight
0,72,72,9
1,72,116,2
2,72,127,6
3,72,128,3
4,72,146,1


## Create the network

In [5]:
# creating an empty graph object
graph_object = nx.Graph()

In [6]:
# iterating through the DataFrame to add edges
for _, edge in df_graph.iterrows():
    graph_object.add_edge(edge['start_station_id'], 
                          edge['end_station_id'],
                          weight=edge['weight'])

### Compare Degree Centrality & Page Rank & Betweenness Centrality

In [8]:
# Creating a list of pagerank, betweenness centrality, degree centrality
measures = [nx.pagerank(graph_object), 
            nx.betweenness_centrality(graph_object, weight='weight'), 
            nx.degree_centrality(graph_object)]

# Creating the correlation DataFrame
cor = pd.DataFrame.from_records(measures)

# Calculating the correlation
heatmap(cor.T.corr())

NameError: name 'heatmap' is not defined

In [155]:
References:
# https://www.kaggle.com/mmmarchetti/game-of-thrones-network-analysis
# https://campus.datacamp.com/courses/introduction-to-network-analysis-in-python/introduction-to-networks?ex=4